In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [28]:
train_df = pd.read_csv('data/train ori.csv')
test_df = pd.read_csv('data/test ori.csv')

In [29]:
train_df.columns

Index(['waktu_setempat', 'id_jalan', 'id_titik_mulai', 'id_titik_akhir',
       'rerata_kecepatan'],
      dtype='object')

In [30]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398648 entries, 0 to 398647
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   waktu_setempat    398648 non-null  object 
 1   id_jalan          398648 non-null  int64  
 2   id_titik_mulai    398648 non-null  int64  
 3   id_titik_akhir    398648 non-null  int64  
 4   rerata_kecepatan  398648 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 15.2+ MB


In [13]:
train_df['waktu_setempat'] = pd.to_datetime(train_df['waktu_setempat']).dt.tz_localize(None)
test_df['waktu_setempat'] = pd.to_datetime(test_df['waktu_setempat']).dt.tz_localize(None)

In [14]:
train_df['unique_id'] = train_df['id_jalan'].astype(str) + '_' + train_df['id_titik_mulai'].astype(str) + '_' + train_df['id_titik_akhir'].astype(str)
test_df['unique_id'] = test_df['id_jalan'].astype(str) + '_' + test_df['id_titik_mulai'].astype(str) + '_' + test_df['id_titik_akhir'].astype(str)

In [15]:
# import pandas as pd
# import seaborn as sns
# # import matplotlib.pyplot as plt

# # Assuming you have already loaded the DataFrame as 'train_df'
# sns.set(style="whitegrid")
# plt.figure(figsize=(12, 8))
# sns.heatmap(train_df.corr(), annot=True, cmap="coolwarm", fmt=".2f")
# plt.title("Correlation Heatmap for train_df")
# plt.show()

In [16]:
train_df = train_df.rename(columns={'waktu_setempat': 'ds', 'rerata_kecepatan': 'y'})
test_df = test_df.rename(columns={'waktu_setempat': 'ds'})

In [17]:
columns_to_drop = ['id_jalan', 'id_titik_mulai', 'id_titik_akhir']
train_df.drop(columns=columns_to_drop, inplace=True)
test_df.drop(columns=columns_to_drop, inplace=True)

In [21]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398648 entries, 0 to 398647
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   ds         398648 non-null  datetime64[ns]
 1   y          398648 non-null  float64       
 2   unique_id  398648 non-null  category      
dtypes: category(1), datetime64[ns](1), float64(1)
memory usage: 6.9 MB


In [19]:
train_df['unique_id'] = train_df['unique_id'].astype('category')
test_df['unique_id'] = test_df['unique_id'].astype('category')

In [20]:
train = train_df.loc[train_df['ds'] < '2020-02-17']
valid = train_df.loc[train_df['ds'] >= '2020-02-17']

In [22]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

models = [RandomForestRegressor(random_state=0, n_estimators=100),
        ExtraTreesRegressor(random_state=0, n_estimators=100)]

In [24]:
from mlforecast import MLForecast
# from mlforecast.target_transforms import Differences
from numba import jit
from window_ops.expanding import expanding_mean
from window_ops.rolling import rolling_mean

@jit(nopython=True)
def diff(x, lag):
    x2 = np.full_like(x, np.nan)
    for i in range(lag, len(x)):
        x2[i] = x[i] - x[i-lag]
    return x2

# Create the lag transforms using rolling window and mean
lag_transforms = {
    1: [(lambda x: x.rolling(window=3).mean()), (lambda x: x.rolling(window=7).mean()),
        (lambda x: x.rolling(window=24).mean()), (diff, 1), (diff, 7)],
    24: [(lambda x: x.diff(24)), (diff, 24)]
}

model = MLForecast(models=models,
                   freq='H',
                   lags=[1, 24, 48],
                   lag_transforms=lag_transforms,
                   date_features=['dayofweek'],
                   num_threads=4)

In [25]:
model.fit(train, id_col='unique_id', time_col='ds', target_col='y', static_features=[])

p = model.predict(horizon=90, dynamic_dfs=[valid[['unique_id', 'ds']]])
p = p.merge(valid[['unique_id', 'ds', 'y']], on=['unique_id', 'ds'], how='left')

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mnon-precise type pyobject[0m
[0m[1mDuring: typing of argument at c:\Users\Fikri\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlforecast\core.py (74)[0m
[1m
File "c:\Users\Fikri\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlforecast\core.py", line 74:[0m
[1mdef _as_tuple(x):
    <source elided>

[1m@njit(nogil=True)
[0m[1m^[0m[0m 

This error may have been caused by the following argument(s):
- argument 4: [1mCannot determine Numba type of <class 'function'>[0m 

This error may have been caused by the following argument(s):
- argument 4: [1mCannot determine Numba type of <class 'function'>[0m


In [179]:
model.preprocess(train, id_col='unique_id', time_col='ds', target_col='y', static_features=[])

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mnon-precise type pyobject[0m
[0m[1mDuring: typing of argument at c:\Users\Fikri\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlforecast\grouped_array.py (16)[0m
[1m
File "c:\Users\Fikri\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlforecast\grouped_array.py", line 16:[0m
[1m# %% ../nbs/grouped_array.ipynb 2
[1m@njit(nogil=True)
[0m[1m^[0m[0m 

This error may have been caused by the following argument(s):
- argument 4: [1mCannot determine Numba type of <class 'function'>[0m 

This error may have been caused by the following argument(s):
- argument 4: [1mCannot determine Numba type of <class 'function'>[0m 

This error may have been caused by the following argument(s):
- argument 4: [1mCannot determine Numba type of <class 'function'>[0m 

This error may have been caused by the following argument(s):
- argument 4: [1mCannot determine Numba type of <class 'function

In [ ]:
model.fit(train, id_col='unique_id', time_col='ds', target_col='y',
            static_features=[], max_horizon=90)

NameError: name 'model' is not defined

In [ ]:
pd.Series(model.models_['RandomForestRegressor'].feature_importances_, index=model.ts.features_order_).sort_values(ascending=False).plot.bar(
            figsize=(1280/96,720/96), title='RandomForestRegressor Feature Importance', xlabel='Features', ylabel='Importance')

In [ ]:
def wmape(y_true, y_pred):
    return np.abs(y_true - y_pred).sum() / np.abs(y_true).sum()

print(f"WMAPE RandomForestRegressor: {wmape(p['y'], p['RandomForestRegressor'])}\nWMAPE ExtraTreesRegressor: {wmape(p['y'], p['ExtraTreesRegressor'])}")

In [ ]:
for model in ['RandomForestRegressor', 'ExtraTreesRegressor']:
    fig,ax = plt.subplots(2,1, figsize=(1280/96, 720/96))
    for ax_, family in enumerate(['MEATS', 'PERSONAL CARE']):
        p.loc[p['unique_id'] == family].plot(x='ds', y='y', ax=ax[ax_], label='y', title=family, linewidth=2)
        p.loc[p['unique_id'] == family].plot(x='ds', y=model, ax=ax[ax_], label=model)
        ax[ax_].set_xlabel('Date')
        ax[ax_].set_ylabel('Sales')
        fig.tight_layout()

In [ ]:
## Train - Test Split
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Function to calculate sMAPE
def smape(actual, forecast):
    return 100 * np.mean(2 * np.abs(forecast - actual) / (np.abs(actual) + np.abs(forecast)))

X = train_df.drop(columns=['rerata_kecepatan'])
# X = train_df.drop(columns=['rerata_kecepatan', 'lanes', 'lanes_forward', 'lanes_backward'])
# X = train_df.drop(columns=['rerata_kecepatan', 'busway', 'lanes_forward', 'lanes_backward'])

y = train_df['rerata_kecepatan']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
# Linear Regression
linear_reg_model = LinearRegression()
linear_reg_model.fit(X_train, y_train)
y_pred_linear_reg = linear_reg_model.predict(X_test)
smape_linear_reg = smape(y_test, y_pred_linear_reg)
print("Linear Regression sMAPE:", smape_linear_reg)

Linear Regression sMAPE: 13.136204649322124


In [ ]:
from xgboost import XGBRegressor

# XGBRegressor
xgb_model = XGBRegressor()
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
smape_xgb = smape(y_test, y_pred_xgb)
print("XGBoost sMAPE:", smape_xgb)

XGBoost sMAPE: 7.047006110646537


In [ ]:
# KNeighborsRegressor
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split

# Create and train the KNeighborsRegressor model
knn_model = KNeighborsRegressor(n_neighbors=5, weights='uniform')  # You can change the hyperparameters here
knn_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_knn = knn_model.predict(X_test)

# Calculate sMAPE
smape_knn = smape(y_test, y_pred_knn)

# Print the sMAPE score
print("KNeighborsRegressor sMAPE:", smape_knn)

KNeighborsRegressor sMAPE: 10.38237706814166


In [ ]:
# RandomForestRegressor
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Create and train the RandomForestRegressor model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)  # You can change the hyperparameters here
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = rf_model.predict(X_test)

# Calculate sMAPE
smape_rf = smape(y_test, y_pred_rf)

# Print the sMAPE score
print("RandomForestRegressor sMAPE:", smape_rf)

In [ ]:
# # Optuna
# import optuna
# import xgboost as xgb
# from sklearn.metrics import mean_squared_error

# # Define the objective function for Optuna
# def objective(trial):
#     # Define the hyperparameters to be optimized
#     params = {
#         'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
#         'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 1.0),
#         'max_depth': trial.suggest_int('max_depth', 3, 10),
#         'subsample': trial.suggest_float('subsample', 0.1, 1.0),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
#     }

#     # Create the XGBoost model with the suggested hyperparameters
#     model = xgb.XGBRegressor(**params)

#     # Train the model
#     model.fit(X_train, y_train)

#     # Make predictions on the test set
#     y_pred = model.predict(X_test)

#     # Calculate the sMAPE
#     smape_score = smape(y_test, y_pred)

#     return smape_score

# # Create an Optuna study object
# study = optuna.create_study(direction='minimize')

# # Start the optimization process
# study.optimize(objective, n_trials=30)

# # Get the best hyperparameters found by Optuna
# best_params = study.best_params

# # Create the XGBoost model with the best hyperparameters
# xgb_model = xgb.XGBRegressor(**best_params)

# # Train the model on the full training data
# xgb_model.fit(X_train, y_train)

# # Make predictions on the test set
# y_pred_xgb = xgb_model.predict(X_test)

# # Calculate sMAPE
# smape_xgb = smape(y_test, y_pred_xgb)

# # Print the results
# print("Best Hyperparameters:", best_params)
# print("XGBoost sMAPE:", smape_xgb)

In [ ]:
# n_estimators = [5,20,50,100] # number of trees in the random forest
# max_features = ['auto', 'sqrt'] # number of features in consideration at every split
# max_depth = [int(x) for x in np.linspace(10, 120, num = 12)] # maximum number of levels allowed in each decision tree
# min_samples_split = [2, 6, 10] # minimum sample number to split a node
# min_samples_leaf = [1, 3, 4] # minimum sample number that can be stored in a leaf node
# bootstrap = [True, False] # method used to sample data points

# random_grid = {'n_estimators': n_estimators,
# 'max_features': max_features,
# 'max_depth': max_depth,
# 'min_samples_split': min_samples_split,
# 'min_samples_leaf': min_samples_leaf,
# 'bootstrap': bootstrap}

In [ ]:
# ## Importing Random Forest Classifier from the sklearn.ensemble
# from sklearn.ensemble import RandomForestRegressor
# rf = RandomForestRegressor()

# from sklearn.model_selection import RandomizedSearchCV
# rf_random = RandomizedSearchCV(estimator = rf,param_distributions = random_grid,
#             n_iter = 100, cv = 5, verbose=2, random_state=35, n_jobs = -1)

# rf_random.fit(X_train, y_train)

# print ('Random grid: ', random_grid, '\n')
# print ('Best Parameters: ', rf_random.best_params_, ' \n')

In [ ]:
# randmf = RandomForestRegressor(n_estimators = 100, min_samples_split = 10, min_samples_leaf = 1, max_features = 'auto', max_depth = 110, bootstrap = True) 
# randmf.fit(X_train, y_train)

# y_pred_rf_random = randmf.predict(X_test)

# smape_rf_random = smape(y_test, y_pred_rf_random)

# print("RandomForestRegressor sMAPE:", smape_rf_random)

In [ ]:
id_series = test_df['id']
id_df = pd.DataFrame({'id': id_series})

test_df.drop(columns=['id'], inplace=True)
# test_df.drop(columns=['id', 'lanes', 'lanes_forward', 'lanes_backward'], inplace=True)
# test_df.drop(columns=['id', 'busway', 'lanes_forward', 'lanes_backward'], inplace=True)

In [ ]:
# y_results = xgb_model.predict(test_df)
# y_results_df = pd.DataFrame({'rerata_kecepatan': y_results})

# # Concatenate id_df and y_results_df horizontally
# result_df = pd.concat([id_df, y_results_df], axis=1)
# print(result_df)

# # Store the concatenated DataFrame to a CSV file
# result_df.to_csv('Data/submission.csv', index=False)

# print("Data saved to 'data/submission.csv'")

In [ ]:
y_results = rf_model.predict(test_df)
y_results_df = pd.DataFrame({'rerata_kecepatan': y_results})

# Concatenate id_df and y_results_df horizontally
result_df = pd.concat([id_df, y_results_df], axis=1)
print(result_df)

# Store the concatenated DataFrame to a CSV file
result_df.to_csv('Data/submission rf.csv', index=False)

print("Data saved to 'data/submission rf.csv'")

            id  rerata_kecepatan
0            0         42.977407
1            1         37.576141
2            2         37.385447
3            3         44.557172
4            4         27.576288
...        ...               ...
127484  127484         32.117302
127485  127485         37.047479
127486  127486         41.292769
127487  127487         34.665056
127488  127488         44.663126

[127489 rows x 2 columns]
Data saved to 'data/submission rf.csv'


In [ ]:
# y_results_randmf = randmf.predict(test_df)

# y_results_rf_random_df = pd.DataFrame({'rerata_kecepatan': y_results_randmf})

# # Concatenate id_df and y_results_df horizontally
# result_df = pd.concat([id_df, y_results_rf_random_df], axis=1)
# print(result_df)

# # Store the concatenated DataFrame to a CSV file
# result_df.to_csv('Data/submission rf random search.csv', index=False)

# print("Data saved")